#### IMPORTS

In [5]:
import os

def find_r_installation():
    possible_paths = []

    # Add standard paths for Windows
    if os.getenv('ProgramFiles'):
        possible_paths.append(os.path.join(os.getenv('ProgramFiles'), 'R'))
    if os.getenv('ProgramFiles(x86)'):
        possible_paths.append(os.path.join(os.getenv('ProgramFiles(x86)'), 'R'))
    if os.getenv('LOCALAPPDATA'):
        possible_paths.append(os.path.join(os.getenv('LOCALAPPDATA'), 'Programs', 'R'))
    
    # Add standard path for Unix-like systems
    if os.getenv('HOME'):
        possible_paths.append(os.path.join(os.getenv('HOME'), 'R'))

    for path in possible_paths:
        if os.path.exists(path):
            for root, dirs, files in os.walk(path):
                if 'bin' in dirs:
                    if 'R.exe' in os.listdir(os.path.join(root, 'bin')):  # Check specifically for Windows
                        return os.path.join(root)
                    if 'R' in os.listdir(os.path.join(root, 'bin')):  # Check for Unix-like systems
                        return os.path.join(root)

    return None

r_path = find_r_installation()
if r_path:
    print(f"R is installed at: {r_path}")
else:
    print("R installation not found.")

os.environ['R_HOME'] = r_path

R is installed at: C:\Users\maceo.valente\AppData\Local\Programs\R\R-4.3.2


In [8]:
import rpy2.robjects as ro
from rpy2.robjects.packages import importr
from rpy2.robjects.vectors import DataFrame, IntVector, StrVector
from rpy2.robjects import pandas2ri

In [10]:
# Activation de la conversion automatique entre pandas DataFrame et R DataFrame
pandas2ri.activate()
# Importation des packages R
sticsOnR = importr('SticsOnR')
sticsRFiles = importr('SticsRFiles')

### Fonctions STICS

In [ ]:
# Function to create an usm
def def_usm(settings):
    # Convertir le dictionnaire de paramètres en DataFrame R
    df_settings = ro.DataFrame(settings)
    # Instancier l'USM avec les paramètres (exemple)
    usm = sticsOnR.create_usm(settings)
    return usm

# Function to run an usm, and save the results
def lancer_simulation(usm):
    # Lancer la simulation (code R spécifique ici)
    resultats = sticsOnR.run_usm(usm)
    # Convertir les résultats R en DataFrame pandas
    df_resultats = pandas2ri.rpy2py(resultats)
    return df_resultats

### Settings USM STICS

In [11]:
# Variables python contenant les paramètres
p_usm = {
    'param1': 1,
    'param2': 2,
    # ajoutez vos paramètres ici
}

In [25]:
# # Charger le script R depuis un fichier
# os.chdir("C:/Users/maceo.valente/Documents/Automatisation/TEST_MVA/TSAgriPV")
# with open("C:/Users/maceo.valente/Documents/Automatisation/TEST_MVA/TSAgriPV/R_related/testVigne.R", 'r') as f:
#     r_script = f.read()
# ro.r(r_script)

v Using stics "modulostics" (exe: "stics_modulo.exe")
[1] "Lang_Romain_1"
[1] "[05/06/24]-[16:37:02] INFO - Files generated :"                                                                         
[2] "[05/06/24]-[16:37:02] INFO - \tC:\\Users\\maceo.valente\\Documents\\Automatisation\\exempleVigne\\mod_bLang_Romain_1.sti"
[3] "[05/06/24]-[16:37:02] INFO - \tC:\\Users\\maceo.valente\\Documents\\Automatisation\\exempleVigne\\modhistory.sti"        
v All usms ran successfully!
[1] 9.18
[1] 11.87215
v Using stics "modulostics" (exe: "stics_modulo.exe")
[1] "Lang_Romain_1"
[1] "[05/06/24]-[16:37:06] INFO - Files generated :"                                                                         
[2] "[05/06/24]-[16:37:06] INFO - \tC:\\Users\\maceo.valente\\Documents\\Automatisation\\exempleVigne\\mod_bLang_Romain_1.sti"
[3] "[05/06/24]-[16:37:06] INFO - \tC:\\Users\\maceo.valente\\Documents\\Automatisation\\exempleVigne\\modhistory.sti"        
v All usms ran successfully!
v Using stic

R[write to console]: In addition: 
R[write to console]: Warning message:

R[write to console]: In dir.create("weatherFiles/") :
R[write to console]:  'weatherFiles' already exists

